In [1]:
import matplotlib.pyplot as plt
#from wordcloud import WordCloud

In [2]:
from nltk.tokenize import TweetTokenizer
from gensim import corpora, models, matutils
import numpy as np
from sklearn import metrics
from operator import itemgetter
from collections import Counter
import json
import gzip
from pprint import pprint
import os
import re

In [33]:
def read_tweets_from_file(category, results_path):
    switcher = {
        "nw": "nw.txt",
        "food": "food.txt",
        "politics": "politics.txt",
        "wellness": "wellness.txt",
        "entertainment": "entertainment.txt",
        "election": "election.txt",
        "nasa":"nasa.txt",
        "addele" : "addele.txt",
        "sport" : "sport.txt"
        }
    file_name = switcher.get(category)
    with open(results_path + file_name, 'rb') as f:
        tweets = f.read().splitlines()
    
    f.close()
    return tweets

In [4]:
print 'hello' 

hello


In [5]:
%matplotlib inline
import os
#import wordcloud
import matplotlib.pyplot as plt

In [37]:
def Clustering_Tweets(category, path, exp, cluster_size, dup_removed=0):
    
    #plt.close('all')
    #fig, axs = plt.subplots(ncols=cluster_size, sharey=True, figsize=(10, 5))

    tweets_all = read_tweets_from_file(category, path)
    tweet_and_id = []
    
    for line in tweets_all:
        try:
            #print line
            tid = line.split(' ', 1)[0]
            ttext = line.split(' ', 1)[1].replace('\r','')
            ttuple = tid,ttext
            tweet_and_id.append(ttuple)
        except IndexError:
            print line
            pass
        continue
            
    #print tweet_and_id
    #return
    tweets = []
    ref = []
    for i,e in enumerate(tweet_and_id):
        if e[1] not in tweets:
            tweets.append(e[1])
            stuple = i,e[0],e[1]
            ref.append(stuple)
    #print ref
    #print tweets[31]
    #print ref[31]
    #return none
    '''if (int(exp)==2):
        #removing duplicate tweets
        #tweets_all = read_tweets_from_file(category, path)
        #tweets = set(tweets_all)
        
        seen = []
        ref = []
        for i,e in enumerate(tweet_and_id):
            if e[1] not in tweets:
                tweets.append(e[1])
                stuple = i,e
                ref.append(stuple)
        
    else:
        tweets = read_tweets_from_file(category, path)'''
    
    stoplist = set([line.replace("\n","") for line in open('/home/ynh3/Data/stopword-list2.txt')])
    pun = [',', '-','.', ':', '(', ')', '--', ';', '...', 'today','follow','stats','can', 'say', 'says', 'will','may', 'must', 'us', 'via','a','the', 'rt', 'gg', 'gt', 'lt', 'la', 'de', 'te', 'lol', 'follow', 'followers', 'unfollow', 'unfollowers', 'unfollower', 'follower']

    with open('/home/ynh3/Data/common_list.txt','rt') as myfile:
        common_word = myfile.read().splitlines()
    #len(common_word)

    #Tokenize
    tknzr = TweetTokenizer()   
    tweet_count = 0
    tweet_tokens=[]
    tweet_list=[]
    i = 0

    for line in tweets:

        new_l = tknzr.tokenize(line)
        lower_l = [e.lower() for e in new_l]
        new_ll = [e for e in lower_l if e not in stoplist and e not in common_word and e not in pun and len(e) > 2 and e.isalpha() and 'http' not in e and '@' not in e]
        
        #removing duplicate after tokenizing
        if (int(exp)==3):
            if (new_ll not in tweet_tokens):
                if len(new_ll) > 0:
                    tweet_tokens.append(new_ll)
                    tweet_count += 1
                    line_id = str(ref[i][1]) + " " + str(line) 
                    tweet_list.append(line_id)

        else:
            if len(new_ll) > 0:
                tweet_tokens.append(new_ll)
                tweet_count += 1
                line_id = str(ref[i][1]) + " " + str(line) 
                tweet_list.append(line_id)
            
        i += 1


    print "Done with tokenizations. tweets count: " , tweet_count
    
    dictionary = corpora.Dictionary(tweet_tokens)
    print(dictionary)

    corpus = [dictionary.doc2bow(t) for t in tweet_tokens]

    n_cluster = cluster_size
    
    # LSI computing
    if (int(exp)==1):
        lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=n_cluster) # initialize an LSI transformation
        corpus_lsi = lsi[corpus]
    else:
        tfidf = models.TfidfModel(corpus)
        corpus_tfidf = tfidf[corpus]


        lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=n_cluster) # initialize an LSI transformation
        corpus_lsi = lsi[corpus_tfidf]

    '''for t in range(lsi.num_topics):
        plt.figure()
        s = lsi.show_topic(t,  topn=5)
        print "in the loop -----------------"
        print s
        plt.imshow(wordcloud.WordCloud().fit_words(s))
        plt.axis("off")
        plt.title("Topic #" + str(t))
        plt.show()'''
    
        
    j = 0
    lsi_topic_labels=[]
    for i,doc in enumerate(corpus_lsi):
        a = np.array(doc)
        absA= abs(a)
        if (len(a)==0):
            cluster_id=0
            prob_value=0.0
            j=j+1
        else:
            try:
                topicmax = absA.argmax(axis=0)
            except ValueError:
                print "ValueError at"
                print i
            cluster_id = a[topicmax.item(1)][0]
            prob_value = abs(a[topicmax.item(1)][1])

        triple = i, cluster_id, prob_value
        lsi_topic_labels.append(triple)
        
        '''l = lsi.print_topic(cluster_id,topn=10)
        
        ##trying new topic words instead of following three lines
        #prob_list = [(abs(float(a[:5])),a[6:])  for a in (l.split('+ '))]
        #sorted(prob_list)
        #t_words = [w[1] for w in prob_list]
        
        #t_words'''

        i=i+1

   
    ##For silhouette score
    labels = [b for a,b,c in lsi_topic_labels]
    np_labels = np.asarray(labels)

    # convert corpus to array
    corpusAsMatrix = matutils.corpus2dense(corpus_lsi, num_terms=n_cluster).transpose()

    score = metrics.silhouette_score(corpusAsMatrix, np_labels, metric='cosine')

    print 'silhouette score is: {}'.format(score)
    
    ###MAKING READY FOR sorting, categorizing and writing into file
    temp = sorted(lsi_topic_labels,key=itemgetter(1,2), reverse=True)
    cluster_topic_list = [e for e in temp if float(e[2]) > 0.2]
    #leftovers_list = [tweet_list[e[0]] for e in temp if float(e[2]) <= 0.2]
    #Counter(a,b,c for a,b,c in lsi_topic_labels)

    cluster_list = sorted(cluster_topic_list,key=lambda x: x[1])
    counter = Counter(b for a,b,c in cluster_list)
    counter_sort = counter.most_common(n_cluster)
    cid_sort = [e[0] for e in counter_sort]

    #print 'length of lsi_topic_labels is {}'.format(len(lsi_topic_labels))
    #print 'length of cluster_list is {}'.format(len(cluster_list))
    #print 'length of counter_sort is {}'.format(len(counter_sort))
    print 'n_cluster is {}'.format(n_cluster)
    #print 'cid_sort is {}'.format(len(cid_sort))

    #t = lsi.show_topics()
    #with open(path + category + '_exp1_topics_v1_' + str(cluster_size) + '.txt', 'wt') as myfile:
    #    myfile.write('\n\n'.join(t))
    
    path = path  + "Experiments_set0_json/"
    
    myfile = open(path + category + '_exp' + str(exp) + '_topics_v' + str(dup_removed)+ '_c' + str(cluster_size) + '.txt', 'wt')
    
        
    output_file = open(path + category + '_exp' + str(exp) + '_clusters_v' + str(dup_removed)+ '_c' + str(cluster_size) + '.txt', 'w')

    def jdefault(o):
        return o.__dict__

    json_file = open(path + category + '_exp' + str(exp) + '_v' + str(dup_removed)+ '_c' + str(cluster_size) +'.json', 'w')


    data_list=[]
    data = {}
    second_dic = {}
    count=0
    i=0

    for i in range(0, len(cid_sort)):
        sublist = [e for e in cluster_topic_list if e[1] == cid_sort[i]]
        t_index = int(cid_sort[i])

        topic = lsi.print_topic(t_index,topn=10)
        
        ##topic word preparation
        new_list = [a  for a in (topic.split('+'))]
        new_list = "".join(new_list)

        prob_list = re.findall(r"[-+]?\d*\.\d+|\d+", new_list)

        topic_word = re.findall(r"[^\W\d_]+", new_list)

        topic_tuple = zip(prob_list,topic_word)
        #print topic_tuple

        neg_prob = [float(p) for p in prob_list if float(p) < 0]
        pos_prob = [float(p) for p in prob_list if float(p) > 0]
        #print sum(neg_prob), sum(pos_prob)

        if len(pos_prob) > len(neg_prob) or abs(sum(pos_prob)) > abs(sum(neg_prob)):
            t_words = [w for p,w in topic_tuple if float(p) > 0]
        else:
            t_words = [w for p,w in topic_tuple if float(p) < 0]

        all_t_words = [w for p,w in topic_tuple]
        all_t = ' '.join(all_t_words)

        topic = ' '.join(t_words)
    
        
        string = ' '.join(topic)
        if string == "":
            print i
            print cid_sort[i], t_index
            print topic
            #print sublist

            break
     
        if (int(dup_removed) == 0):
            l = [tweet_list[ss[0]].split(' ', 1) for ss in sublist if float(ss[2]) > 0.48]
        
        else:
            
            t = [tweet_list[ss[0]].split(' ', 1)[1] for ss in sublist if float(ss[2]) > 0.48]
            tweets_unique = set(t)
            ##printing just unique and not duplicated tweets
            l_all = [tweet_list[ss[0]].split(' ', 1) for ss in sublist if float(ss[2]) > 0.48 and tweet_list[ss[0]].split(' ', 1)[1] not in tweet_...]
            temp = set(l_all)
            #temp = sorted(lsi_topic_labels,key=itemgetter(1,2), reverse=True)
            l = sorted(temp, key=lambda x:x[0],reverse=True)

            
        count += len(l)
        myfile.write('Topic # %d' %t_index + ': %s' %topic +'\n')
        #myfile.write('Topic # %d: %s' %t_index %topic +'\n')
        
        output_file.write("\n\n")
        output_file.write('Cluster # %d' %t_index + '-----------------------\n')
        output_file.write('Total items # %d' %len(l) + '-----------------------\n\n')
        output_file.write('Topic: %s' %topic +'\n')
        #output_file.write('Topic: %s' %all_t +'\n')
        ##output_file.write("\n".join(l))
        
        #writing the probability of each tweet for the current topic
        output_file.write("".join("%s %s\n" % (tup[0],tup[1]) for tup in l))
        #output_file.write("".join("%s\n" % tup for tup in l))
        #output_file.write("%d, %s\n" % (int(a),str(b)) for a,b in l)

        #data_list=[]
        data = {}
        topics_dic = {}
        
        
        data['clusters']= [[e[0]] for e in l]
        data['topic']=topic
        #data['tweet']=tweet_str
        
        second_dic['T'+str(i)]=data
        #data_list.append(data)
        #topics_dic['topics'] = data_list

        print('Cluster # %d' %t_index + '-----------------------\n')
        print('Total items # %d' %len(l) + '-----------------------\n\n')
        print('Topic: %s' %topic+'\n')
        #print ", ".join(str(tup) for tup in l)
        print "".join("%s %s\n" % (tup[0], tup[1]) for tup in l)
        #print("\n".join(l))

    #for p in range(0,len(data_list)):
    #    record = data_list[p]
    #    json_data = json.dumps(record, default=jdefault)
    #    json_file.writelines(json_data)
    
    topics_dic['topics'] = second_dic
    json_data = json.dumps(topics_dic, default=jdefault)
    json_file.writelines(json_data)
        
    json_file.close()
    
    print("Total number of tweets is {}".format(count))
    
    output_file.write('Total tweets being clustered is # %d' %count)
    output_file.close()

    #plt.show() 

In [69]:
#######################

In [40]:
Clustering_Tweets("nw","/home/ynh3/Documents/ExS/results/recent-tweets/",1,10,0)



Done with tokenizations. tweets count:  9964
Dictionary(11575 unique tokens: [u'paperless', u'writings', u'yellow', u'jihad', u'mirage']...)
silhouette score is: 0.27301004529
n_cluster is 10
Cluster # 0-----------------------

Total items # 1251-----------------------


Topic: trump president russia flynn fake campaign russian aides know house

832555913852645376 @cnn shoulda read: -trump news conference  -trump news conference -trump news conference -trump news conference -tr https://t.co/zqig9ste2q
831653681326063620 4 'winner,' trump  doing lot of losing @cnn https://t.co/sac4zorxvr the only person who ever said trump was a winner was trump! loser don!
832609940678275073 rt @cnnpolitics: president donald trump calls the treatment of first lady melania trump unfair https://t.co/rezmjniurv https://t.co/y3p98hc
832278980736659456 @cnnpolitics what happened to flynn/trump/russia?? potus got u 2change the narrative! what about investigating trump/russia?? #traitorpotus
832290028550230

In [12]:
Clustering_Tweets("nw","/home/ynh3/Documents/ExS/results/categorized_tweets_enhanced_1h/",1,10,0)


Done with tokenizations. tweets count:  40
Dictionary(217 unique tokens: [u'bomb', u'just', u'diplomacy', u'hands', u'mission']...)
silhouette score is: 0.276387751102
n_cluster is 10
Cluster # 0-----------------------

Total items # 4-----------------------


Topic: women arrested bomb attacks build planning laden allegedly osama idolized

583792863512670209 rt @cnnsitroom: 2 women arrested for allegedly planning to build bomb for us attacks; idolized osama bin laden http://t.co/gvwjztfxke http:
 583795510106009601 rt @cnnsitroom: 2 women were arrested &amp; accused of planning to build a bomb for attacks in the united states: http://t.co/gvwjztxyio http:/
 583794344089354241 rt @cnnbrk: 2 new york women arrested in isis-inspired bomb plot for attacks in the u.s. http://t.co/kq43qxhfsk
 583787628992028673 rt @ajam: prosecutors: two nyc women arrested on terror-related charges http://t.co/8ee1t2mu03

Cluster # 1-----------------------

Total items # 3-----------------------


Topic: re

In [18]:
Clustering_Tweets("nw","/home/ynh3/Documents/ExS/results/categorized_tweets_and_id_2ndweek/",1,10,0)

Done with tokenizations. tweets count:  5455
Dictionary(8793 unique tokens: [u'unscientific', u'foul', u'jihad', u'spiders', u'hanging']...)
silhouette score is: 0.276726603508
n_cluster is 10
Cluster # 0-----------------------

Total items # 253-----------------------


Topic: police shooting officer man south carolina charged murder black back

585573978896527360 rt @yahoonews: white south carolina police officer to be charged with murder in fatal shooting of black man http://t.co/zweeuzmntj
585575425902039043 rt @nprnews: south carolina police officer charged with murder after shooting man in back http://t.co/7aol05ywgq
585578236098322432 rt @owlsasylum: rt @nytimes in a video, a white police officer in south carolina is seen shooting an apparently unarmed black man http://t.
585580693981368323 rt @nytimes: in a video, a white police officer in south carolina is seen shooting an apparently unarmed black man http://t.co/gp2jlg9e16
585583030212886530 "south carolina police officer cha